In [2]:
import torch
import pandas as pd
import numpy as np

In [3]:
#Set random seed
torch.manual_seed(1234)

In [4]:
#CHEMBL canonical SMILES data consists of 51 unique characters: one hot embedding with vocabulary size 51

#Input of size [sequence length, batch size, embedding dimension] = [length of SMILES string, batch dize, 51]
#Output of size [sequence length, batch size, hidden dimension] = [length of SMILES string, batch size, 1024]
lstm = torch.nn.LSTM(input_size = 51, hidden_size = 1024, num_layers = 3, dropout = 0.2)

#Transform outputs to have dimensions of input (from hidden dimension to embedding demension)
linear = torch.nn.Linear(1024, 51)

#Softmax function
softmax = torch.nn.functional.softmax

#Loss function
loss = torch.nn.CrossEntropyLoss()

#Adam optimizer 
optimizer = torch.optim.Adam(list(lstm.parameters()) + list(linear.parameters()), lr=0.001)

In [ ]:
num_epochs = 10

accuracies, max_accuracy = [], 0
for x in range(num_epochs):
    print('Epoch: {}'.format(x))
    for encrypted, original in dataset(num_examples):
        # encrypted.size() = [64]
        lstm_in = embed(encrypted)
        # lstm_in.size() = [64, 51]. This is a 2D tensor, but LSTM expects 
        # a 3D tensor. So we insert a fake dimension.
        lstm_in = lstm_in.unsqueeze(1)
        # lstm_in.size() = [64, 1, 51]
        # Get outputs from the LSTM.
        lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden())
        # lstm_out.size() = [64, 1, 1024]
        # Apply the affine transform.
        scores = linear(lstm_out)
        # scores.size() = [64, 1, 1024], but loss_fn expects a tensor
        # of size [64, 1024, 1]. So we switch the second and third dimensions.
        scores = scores.transpose(1, 2)
        # original.size() = [64], but original should also be a 2D tensor
        # of size [64, 1]. So we insert a fake dimension.
        original = original.unsqueeze(1)
        # Calculate loss.
        loss = loss_fn(scores, original) 
        # Backpropagate
        loss.backward()
        # Update weights
        optimizer.step()
    print('Loss: {:6.4f}'.format(loss.item()))